<a href="https://colab.research.google.com/github/Esantomi/Chatbot/blob/main/twitterChatbot_weatherData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 모듈 import

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import os
os.chdir("/content/gdrive/MyDrive/Colab_Notebooks/트위터봇_코드")

In [6]:
#날짜 관련 모듈
import datetime

#엑셀 파일 여는 모듈
import pandas as pd

# API를 받는 데 필요한 모듈
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json

# 날씨 정보 가져오기

## API 가져오는 법 
1) 설명해주는 문서 파일 다운 받기 <br>
2) 원하는 서비스 부분 찾기

In [7]:
# 기상청_동네예보 조회서비스 decoding key 가져오기
# https://www.data.go.kr/iim/api/selectAPIAcountView.do
service_key = "Ah/PAcOed3J9ZSo5sfX8YALuElqr02D/EZgkIRKTxx75qQCTVaJql1v0Z20AMfP3UC5Miww6RRRIDOHJn4BQiw=="

today = datetime.datetime.today()
base_date = today.strftime("%Y%m%d") # "20210723" == 기준 날짜
base_time = "1400" # 날씨 값 오늘은 1400 기준으로 가져오겠습니다. (보통 40분 뒤에 업데이트)

nx = "60"  # 경도 좌표 가져오기
ny = "128"

In [9]:
base_date

'20210723'

## 데이터 가져오기

동네예보
- Base_time : 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 (1일 8회)
- API 제공 시간(~이후) : 02:05, 05:05, 08:05, 11:05, 14:05, 17:05, 20:05, 23:05 <br>
※ 예보 생성시간에 따라 API 제공 시간의 변동이 있을 수 있음


In [10]:
def get_weather_data(service_key, base_date, base_time, nx, ny):
    # Setting for URL parsing

    CallBackURL = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'
    
    # 기상청18_동네예보 조회서비스_오픈API활용가이드.docx 참고
    params = '?' + urlencode({
        quote_plus("serviceKey"): service_key,  # 인증키
        quote_plus("numOfRows"): "10",          # 한 페이지 결과 수 // default : 10
        quote_plus("pageNo"): "1",              # 페이지 번호 // default : 1
        quote_plus("dataType"): "JSON",         # 응답자료형식 : XML, JSON
        quote_plus("base_date"): base_date,     # 발표일자 // yyyymmdd
        quote_plus("base_time"): base_time,     # 발표시각 // HHMM, 매 시각 40분 이후 호출
        quote_plus("nx"): nx,                   # 예보지점 X 좌표
        quote_plus("ny"): ny                    # 예보지점 Y 좌표
    })

    final_url = CallBackURL + unquote(params)

    # Python이 URL로 들어가서 데이터를 가져옴
    req = urllib.request.Request(final_url)

    # Get Data from API (가져온 데이터를 파이썬이 읽음)
    response_body = urlopen(req).read() # get bytes data

    # print(response_body)
    # # Convert bytes to json (사람이 읽을 수 있는 형태로 변환)
    data = json.loads(response_body)

    data = data['response']['body']['items']['item']
    
    return data

## 데이터 받기

In [11]:
data = get_weather_data(service_key, base_date, base_time, nx, ny)

In [12]:
data

[{'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'POP',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '0',
  'nx': 60,
  'ny': 128},
 {'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'PTY',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '0',
  'nx': 60,
  'ny': 128},
 {'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'R06',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '0',
  'nx': 60,
  'ny': 128},
 {'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'REH',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '50',
  'nx': 60,
  'ny': 128},
 {'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'S06',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '0',
  'nx': 60,
  'ny': 128},
 {'baseDate': '20210723',
  'baseTime': '1400',
  'category': 'SKY',
  'fcstDate': '20210723',
  'fcstTime': '1800',
  'fcstValue': '1',
  'nx': 60,
  'ny': 128},
 {'baseDate': '202107

## 보기 편하게 바꿔주기

In [13]:
def change_category(category):
    response_key = {'POP' : "강수확률", 'PTY' : "강수형태", 'R06' : "6시간 강수량", 'REH' : "습도", 'S06' : '6시간 신적설', 'SKY':'하늘상태','T3H' : "3시간 기온", "TMN" : "아침 최저기온", "TMX" : "낮 최고기온", "UUU" : "풍속(동서성분)", "VVV" : "풍속(남북성분)", "WAV":"파고", "VEC" : "풍향", "WSD" : "풍속"}
    return response_key[category]


# Result
res = pd.DataFrame(data)

res['category'] = res['category'].apply(change_category)

res

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20210723,1400,강수확률,20210723,1800,0,60,128
1,20210723,1400,강수형태,20210723,1800,0,60,128
2,20210723,1400,6시간 강수량,20210723,1800,0,60,128
3,20210723,1400,습도,20210723,1800,50,60,128
4,20210723,1400,6시간 신적설,20210723,1800,0,60,128
5,20210723,1400,하늘상태,20210723,1800,1,60,128
6,20210723,1400,3시간 기온,20210723,1800,32,60,128
7,20210723,1400,풍속(동서성분),20210723,1800,0,60,128
8,20210723,1400,풍향,20210723,1800,175,60,128
9,20210723,1400,풍속(남북성분),20210723,1800,1.2,60,128


## 기온과 비 오는 지 여부 확인하기

In [14]:
# 이 값들 중에서 PTY(강수형태)와 T3H(3시간 기온) 정보를 통해 비가 오는지, 기온은 어떠한지 알 수 있음
# PTY : 0-없음 1-비 2-비/눈(진눈깨비) 3-눈 4-소나기

data = dict()
data['date'] = res.iloc[0]['baseDate']

## 기온 확인

In [15]:
temparature = res.loc[res["category"]=='3시간 기온', 'fcstValue'].iloc[0]

data['temperature'] = temparature
data

{'date': '20210723', 'temperature': '32'}

## 날씨 여부 확인

강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4), 빗방울(5), 빗방울/눈날림(6), 눈날림(7) <br>
여기서 비/눈은 비와 눈이 섞여 오는 것을 의미 (진눈개비)


In [16]:
weather_data = dict()

weather_code = res.loc[res["category"]=='강수형태', 'fcstValue'].iloc[0]
if weather_code == '1':
    weather_state = '비'
elif weather_code == '2':
    weather_state = '비/눈'
elif weather_code == '3':
    weather_state = '눈'
elif weather_code == '4':
    weather_state = '소나기'
else:
    weather_state = '없음'

### 함수로 만들기

In [17]:
def get_PTY(res):
    ''' 
    강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4), 빗방울(5), 빗방울/눈날림(6), 눈날림(7) <br>
    여기서 비/눈은 비와 눈이 섞여 오는 것을 의미 (진눈개비)
    '''
    weather_data = dict()

    weather_code = res.loc[res["category"]=='강수형태', 'fcstValue'].iloc[0]
    if weather_code == '1':
        weather_state = '비'
    elif weather_code == '2':
        weather_state = '비/눈'
    elif weather_code == '3':
        weather_state = '눈'
    elif weather_code == '4':
        weather_state = '소나기'
    else:
        weather_state = '없음'
    return weather_state

In [18]:
weather_state = get_PTY(res)
data['weather'] = weather_state

In [19]:
data

{'date': '20210723', 'temperature': '32', 'weather': '없음'}

## 전처리까지 완료한 최종 코드

In [20]:
def get_final_weather_data(service_key, base_date, base_time, nx, ny): 
        
    # 1) 데이터 불러오기 
    data = get_weather_data(service_key, base_date, base_time, nx, ny)
    res = pd.DataFrame(data)
    
    # 2) 데이터 전처리
    # 2-1) 카테고리 보기 편하게 바꾸기
    res['category'] = res['category'].apply(change_category)
    
    
    # 3) 원하는 정보 뽑기 
    
    # 1) 날짜
    result = dict()
    result['date'] = res.iloc[0]['baseDate']
    
    # 2) 기온
    temparature = res.loc[res["category"]=='3시간 기온', 'fcstValue'].iloc[0]
    result['temperature'] = temparature
    
    # 3) 강수량
    ''' 
    강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4), 빗방울(5), 빗방울/눈날림(6), 눈날림(7) <br>
    여기서 비/눈은 비와 눈이 섞여 오는 것을 의미 (진눈개비)
    '''

    weather_state = get_PTY(res)
    result['weather'] = weather_state
    
    return result

In [21]:
result = get_final_weather_data(service_key, base_date, base_time, nx, ny)

In [22]:
result

{'date': '20210723', 'temperature': '32', 'weather': '없음'}

# 트위터봇 

트위터봇 페이지 : https://twitter.com/osd_weather_BOT

In [23]:
!pip install tweepy

In [24]:
# 트위터 api 사용 모듈 가져오기
import tweepy

In [29]:
def send_twitter(tweet):
    
    consumer_key = 'G3IuuwV43XiFjciRKMHUHPRnP'
    consumer_secret = 'Lfet5uPwvG1qCLCuCxmhvk8AOcjIdcuaNVuOdUXWmpP3wjIkw2' # Secret Key
    access_token = '1364114536626749441-0ILRGxLotKxyx20AJOCCXBwNWRyoLa'
    access_token_secret = '1Oam6PQbygE41ZPdACHnjKaZ1H0zxSL1LcmvoG1PVsTw2'

    # 인증키 값들 보내주기
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret) # 인증 핸들러 (consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)  # access_token, access_token_secret

    # 트위터 연동
    try :
        api = tweepy.API(auth)
        print('트위터 연동 Success')
    except Exception : 
        print('트위터 연동 Failure')
    
    api.update_status(tweet)
    print("전송 완료")

In [30]:
tweet = '하나 둘 셋 야!'
send_twitter(tweet)

트위터 연동 Success
전송 완료


## 사진도 함께 전송

In [31]:
consumer_key = 'G3IuuwV43XiFjciRKMHUHPRnP'
consumer_secret = 'Lfet5uPwvG1qCLCuCxmhvk8AOcjIdcuaNVuOdUXWmpP3wjIkw2' # Secret Key
access_token = '1364114536626749441-0ILRGxLotKxyx20AJOCCXBwNWRyoLa'
access_token_secret = '1Oam6PQbygE41ZPdACHnjKaZ1H0zxSL1LcmvoG1PVsTw2'

# 인증키 값들 보내주기
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) # consumer_key, consumer_secret
auth.set_access_token(access_token, access_token_secret)  # access_token, access_token_secret

try :
    api = tweepy.API(auth)
    print('트위터 연동 Success')
except Exception : 
    print('트위터 연동 Failure')

트위터 연동 Success


In [32]:
# 사진도 같이 업로드!

tweet = '!야 셋 둘 하나'                                  # tweet할 메시지
media_ids = []                                            # 빈 리스트 생성
res = api.media_upload('./weird.jpg')                     # response. 사진 업로드해서 res에 할당
media_ids.append(res.media_id)                            # 리스트에 이미지 주소 추가
api.update_status(media_ids = media_ids, status = tweet)  # 이미지, 트윗 올리기

Status(_api=<tweepy.api.API object at 0x7f318601e4d0>, _json={'created_at': 'Fri Jul 23 08:49:21 +0000 2021', 'id': 1418493423116603392, 'id_str': '1418493423116603392', 'text': '!야 셋 둘 하나 https://t.co/9SKQ5bhEpw', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1418493421786968064, 'id_str': '1418493421786968064', 'indices': [10, 33], 'media_url': 'http://pbs.twimg.com/media/E6-AQGnXMAAQTb-.jpg', 'media_url_https': 'https://pbs.twimg.com/media/E6-AQGnXMAAQTb-.jpg', 'url': 'https://t.co/9SKQ5bhEpw', 'display_url': 'pic.twitter.com/9SKQ5bhEpw', 'expanded_url': 'https://twitter.com/firmbarrentree/status/1418493423116603392/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 1000, 'h': 750, 'resize': 'fit'}, 'large': {'w': 1000, 'h': 750, 'resize': 'fit'}, 'small': {'w': 680, 'h': 510, 'resize': 'fit'}}}]}, 'extended_entities': {'media': [{'id': 1418493421786968064, 'id_str'

## 사진 업로드까지 합친 함수

In [35]:
def send_twitter_with_image(tweet, image_path):
    
    # 각자 트위터 값으로 바꿔주기
    consumer_key = 'G3IuuwV43XiFjciRKMHUHPRnP'
    consumer_secret = 'Lfet5uPwvG1qCLCuCxmhvk8AOcjIdcuaNVuOdUXWmpP3wjIkw2' # Secret Key
    access_token = '1364114536626749441-0ILRGxLotKxyx20AJOCCXBwNWRyoLa'
    access_token_secret = '1Oam6PQbygE41ZPdACHnjKaZ1H0zxSL1LcmvoG1PVsTw2'

    # 인증키 값들 보내주기
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)  # consumer_key, consumer_secret
    auth.set_access_token(access_token, access_token_secret)   # access_token, access_token_secret

    try :
        api = tweepy.API(auth)
        print('트위터 연동 Success')
    except Exception : 
        print('트위터 연동 Failure')
    
    # tweet = tweet
    media_ids = []                      # 빈 리스트 생성
    res = api.media_upload(image_path)  # response. 사진 업로드해서 res에 할당
    media_ids.append(res.media_id)      # 리스트에 이미지 주소 추가
    api.update_status(media_ids=media_ids,status=tweet)  # 이미지, 트윗 올리기
    
    print("전송완료")

In [36]:
tweet = '야 야! 하나 셋' 
image_path = './weird.jpg'  # 사진의 경로 
send_twitter_with_image(tweet, image_path)

트위터 연동 Success
전송완료


# 최종
##  공공데이터로 불러온 날씨도 넣어서 완성해보자!

In [37]:
def change_rain_data(result):
    
    weather = result['weather']
    
    if weather =='비':
        rain_data = '비가 올 것으로 예상됩니다.'
    elif weather =='비/눈':
        rain_data = '비나 눈이 올 것으로 예상됩니다.'
    elif weather == '눈':
        rain_data = '눈이 올 것으로 예상됩니다.'
    elif weather == '소나기':
        rain_data = '소나기가 올 것으로 예상됩니다.'
    else:
        rain_data = '비 소식은 없습니다.'
    
    return rain_data

In [38]:
change_rain_data(result)

'비 소식은 없습니다.'

In [39]:
result

{'date': '20210723', 'temperature': '32', 'weather': '없음'}

In [41]:
def final_code(service_key, base_date, base_time, nx, ny):
    
    result = get_final_weather_data(service_key, base_date, base_time, nx, ny)
    rain_data = change_rain_data(result)
    
    tweet = '고래 어깨가 무친 것은 ' + result['temperature'] + '시절만큼의 힘을, ' + rain_data
    image_path = './weird.jpg'  # 사진의 경로 
    send_twitter_with_image(tweet, image_path)

In [42]:
final_code(service_key, base_date, base_time, nx, ny)

트위터 연동 Success
전송완료


# 매일 아침 8시에 트위터로 날씨 보내기 (스케쥴링)

## 여기서부터는 실행하지 말아 주세요! 

In [ ]:
# 필요 패키지 가져오기
import schedule
from time import sleep

In [ ]:
# 매일 xx시 xx분 업로드 되게끔 설정
schedule.every().day.at("08:10").do(final_code)
while True:
    schedule.run_pending()
    sleep(30) 

# 과제

# 실시간 미세먼지 데이터 가져와서 트위터로 보내기

* 각 구의 실시간 미세먼지 데이터를 가져와서 이를 트위터로 보내보는 것이 실습 과제입니다!



**웹페이지에서 해야 하는 내용**

1) 회원가입을 한다. 

2) 내가 받고자 하는 공공데이터 API를 찾는다

3) 활용신청을 해  API Key를 받는다

4) 내가 요청하고자 하는 내용을 설명서에서 찾는다

5) 요청하고자 하는 변수를 기록해둔다.

6) 응답받을 내용을 기록해둔다. 

---

**코딩이 필요한 절차** 

7) Request를 한다. 

8) Response를 받는다

9) 6)에서 적어둔 Response의 내용별 CODE를 설명서에서 찾아 해석한다

10) 내가 원하는 정보만 뽑아낸다.